In [1]:

import numpy as np
import supervision as sv
import cv2
print("supervision", sv.__version__)

supervision 0.18.0


In [2]:
from IPython import display
import glob
import os
import numpy as np
from itertools import combinations
import matplotlib.patches as patches
from collections import namedtuple
from collections import defaultdict
import matplotlib.pyplot as plt
import cv2
import sys
import math
import pandas as pd
from PIL import Image, ImageDraw, ImageFont
import argparse
import platform
import shutil
import time
from pathlib import Path
import logging
from typing import Callable, Generator, Optional, Tuple

# logging.getLogger().removeHandler(logging.getLogger().handlers[0])

import torch
import ultralytics

ultralytics.checks()
from ultralytics import YOLO

import supervision as sv
from supervision.draw.color import Color, ColorPalette

print("supervision", sv.__version__)


# import dev
from helpers.annotator import BoxAnnotator
from helpers.polygon_annotator import PolygonZoneAnnotator
from helpers.crossing_line import CrossingLine
from boxex import Boxes
from boxex import Results

import dev

Ultralytics YOLOv8.1.14 🚀 Python-3.11.7 torch-2.2.0+cu121 CUDA:0 (NVIDIA GeForce RTX 3090, 24576MiB)
Setup complete ✅ (16 CPUs, 63.9 GB RAM, 7706.6/9314.0 GB disk)
supervision 0.18.0


In [3]:
!nvidia-smi

Thu Feb 29 10:44:44 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 531.14                 Driver Version: 531.14       CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                      TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3090       WDDM | 00000000:01:00.0  On |                  N/A |
|  0%   53C    P8               36W / 350W|   2885MiB / 24576MiB |      1%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [4]:
import torch
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Mon_Apr__3_17:36:15_Pacific_Daylight_Time_2023
Cuda compilation tools, release 12.1, V12.1.105
Build cuda_12.1.r12.1/compiler.32688072_0
torch:  2.2 ; cuda:  cu121


In [5]:
import os
HOME = os.getcwd()
print(HOME)

e:\Traffic_


In [6]:
from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.1.14 🚀 Python-3.11.7 torch-2.2.0+cu121 CUDA:0 (NVIDIA GeForce RTX 3090, 24576MiB)
Setup complete ✅ (16 CPUs, 63.9 GB RAM, 7706.6/9314.0 GB disk)


In [7]:
from IPython import display
display.clear_output()



In [8]:
SOURCE_VIDEO= "E:/Traffic_/video/hiv.mp4"

In [9]:
from ultralytics import YOLO

model = YOLO('best.pt')

In [10]:
D = model.model.names

In [11]:
vehicle_tracking_ids = []
vehicle_count = {x:0 for x in list(model.model.names.keys())}

In [12]:
def drawing_on_frame_after_detection(frame):
    overlay = frame.copy()
    im0 = frame
    points_x = [[655, 40], [900, 40], [900, 70], [675, 70]]
    points_x = np.array(points_x)
    points_x = points_x.reshape((-1, 1, 2))
    # color = (255, 255, 255)
    thickness = 2
    isClosed = True
    cv2.fillPoly(overlay, [points_x], (0, 255, 0))
    alpha = 0.4
    im0 = cv2.addWeighted(overlay, alpha, frame, 1 - alpha, 0)

    frame = im0
    overlay = frame.copy()
    points_z = [[1267, 40], [1010, 40], [1010, 70], [1247, 70]]
    points_z = np.array(points_z)
    points_z = points_z.reshape((-1, 1, 2))
    # color = (255, 255, 255)
    thickness = 2
    isClosed = True
    cv2.fillPoly(overlay, [points_z], (0, 255, 0))
    alpha = 0.4
    frame = cv2.addWeighted(overlay, alpha, frame, 1 - alpha, 0)

    points = [
        [10, 10],
        [900, 10],
        [925, 30],
        [995, 30],
        [1020, 10],
        [1910, 10],
        [1910, 1070],
        [1020, 1070],
        [995, 1050],
        [925, 1050],
        [900, 1070],
        [10, 1070],
        [10, 10],
    ]
    points = np.array(points)
    points = points.reshape((-1, 1, 2))
    color = (255, 255, 255)
    thickness = 2
    isClosed = False
    cv2.polylines(frame, [points], isClosed, color, thickness)

    points_1 = [[30, 60], [60, 30], [800, 30]]
    points_1 = np.array(points_1)
    points_1 = points_1.reshape((-1, 1, 2))
    color = (255, 255, 255)
    thickness = 1
    isClosed = False
    cv2.polylines(frame, [points_1], isClosed, color, thickness)

    #######-------------------------------------------- 2nd RIGHT LINE -----------------------------------------------#######
    #######-------------------------------------------- 2nd RIGHT LINE -----------------------------------------------#######
    #######-------------------------------------------- 2nd RIGHT LINE -----------------------------------------------#######

    points_2 = [[1120, 30], [1860, 30], [1890, 60]]
    points_2 = np.array(points_2)
    points_2 = points_2.reshape((-1, 1, 2))
    color = (255, 255, 255)
    thickness = 1
    isClosed = False
    cv2.polylines(frame, [points_2], isClosed, color, thickness)

    #######-------------------------------------------- 3nd LEFT LINE -----------------------------------------------#######
    #######-------------------------------------------- 3nd LEFT LINE -----------------------------------------------#######
    #######-------------------------------------------- 3nd LEFT LINE -----------------------------------------------#######

    points_3 = [[500, 50], [650, 50], [670, 80], [900, 80]]
    points_3 = np.array(points_3)
    points_3 = points_3.reshape((-1, 1, 2))
    color = (255, 255, 255)
    thickness = 1
    isClosed = False
    cv2.polylines(frame, [points_3], isClosed, color, thickness)

    #######-------------------------------------------- 3nd RIGHT LINE -----------------------------------------------#######
    #######-------------------------------------------- 3nd RIGHT LINE -----------------------------------------------#######
    #######-------------------------------------------- 3nd RIGHT LINE -----------------------------------------------#######

    points_4 = [[1420, 50], [1270, 50], [1250, 80], [1020, 80]]
    points_4 = np.array(points_4)
    points_4 = points_4.reshape((-1, 1, 2))
    color = (255, 255, 255)
    thickness = 1
    isClosed = False
    cv2.polylines(frame, [points_4], isClosed, color, thickness)

    # cv2.putText(frame, "LOCATION - ", (800,60) ,0,0, 0.75, (255,255,255),2)
    font = cv2.FONT_HERSHEY_SIMPLEX

    # org
    org = (850, 60)
    # fontScale
    fontScale = 0.87
    # Blue color in BGR
    color = (255, 255, 255)
    # Line thickness of 2 px
    thickness = 2
    fontScale = 0.6
    thickness = 1

    #######-------------------------------------------- LATITUDE, LONGITUDE, DATA, TIME, ALTITUDE TEXT -----------------------------------------------#######
    #######-------------------------------------------- LATITUDE, LONGITUDE, DATA, TIME, ALTITUDE TEXT -----------------------------------------------#######
    #######-------------------------------------------- LATITUDE, LONGITUDE, DATA, TIME, ALTITUDE TEXT -----------------------------------------------#######

    font2 = ImageFont.truetype("font/Roboto-Bold.ttf", size=25)
    font3 = ImageFont.truetype("font/Roboto-Light.ttf", size=18)
    font4 = ImageFont.truetype("font/Depot Regular 400.ttf", size=18)
    font5 = ImageFont.truetype("font/Exo-Medium.ttf", size=16)

    frame = Image.fromarray(frame)
    draw = ImageDraw.Draw(frame, "RGBA")
    (x, y) = (500, 500)
    color = "rgb(0, 0, 0)"  # black color

    # Draw a rounded rectangle

    overlay = frame.copy()
    draw2 = ImageDraw.Draw(overlay)
    alpha = 0.5

    logo = cv2.imread("misc" + os.sep + "logo.png", cv2.IMREAD_UNCHANGED)

    draw.rounded_rectangle(
        (1690, 100, 1870, 200), fill=(0, 0, 0, 150), width=2, radius=15
    )
    draw.rounded_rectangle(
        (1690, 210, 1870, 310), fill=(0, 0, 0, 150), width=2, radius=15
    )
    draw.rounded_rectangle(
        (1690, 320, 1870, 420), fill=(0, 0, 0, 150), width=2, radius=15
    )
    draw.rounded_rectangle(
        (1690, 430, 1870, 530), fill=(0, 0, 0, 150), width=2, radius=15
    )

    draw.rounded_rectangle(
        (1580, 980, 1900, 1060), fill=(255, 255, 255, 150), width=2, radius=1
    )

    ht, wd = im0.shape[:2]

    # print("HT:::", ht)
    # print("WD:::", wd)

    # read overlay image
    # img2 = cv2.imread("sunglasses.png", cv2.IMREAD_UNCHANGED)
    ht2, wd2 = logo.shape[:2]

    # extract alpha channel as mask and base bgr images
    bgr = logo[:, :, 0:3]
    mask = logo[:, :, 3]

    # insert bgr into img at desired location and insert mask into black image
    x = 1600
    y = 1000

    bgr_new = im0.copy()
    bgr_new[y : y + ht2, x : x + wd2] = bgr

    mask_new = np.zeros((ht, wd), dtype=np.uint8)
    mask_new[y : y + ht2, x : x + wd2] = mask
    mask_new = cv2.cvtColor(mask_new, cv2.COLOR_GRAY2BGR)

    morph_kernel = np.zeros((3, 3), np.uint8)
    ret, mask_new = cv2.threshold(mask_new, 0, 255, cv2.THRESH_BINARY)

    # mask_new = cv2.dilate(mask_new,morph_kernel,iterations = 1)

    # overlay the base bgr image onto img using mask
    frame = np.where(mask_new == 255, bgr_new, frame)

    frame = Image.fromarray(frame)
    draw = ImageDraw.Draw(frame, "RGBA")
    overlay = frame.copy()
    draw2 = ImageDraw.Draw(overlay)

    # frame = np.array(frame)
    # ------------------------------------------------------------------------------------------------------------------------------------

    # draw.text((1730,120), format((groups[frame_num - 1]["time_stamp"])), fill=(255,255,255), font=font2)
    # draw.text((1760,165), "TIME", fill=(255,255,255), font=font3)

    # draw.text((1720,230), format((groups[frame_num - 1]["date"])), fill=(255,255,255), font=font2)
    # draw.text((1750,275), "DATE", fill=(255,255,255), font=font3)

    # draw.text((1752,340), format((groups[frame_num - 1]["iso"])), fill=(255,255,255), font=font2)
    # draw.text((1760,385), "ISO", fill=(255,255,255), font=font3)

    # draw.text((1745,450), format((groups[frame_num - 1]["shutter"])), fill=(255,255,255), font=font2)
    # draw.text((1748,495), "SHUTTER", fill=(255,255,255), font=font3)

    # draw.text((775,45), format((groups[frame_num - 1]["latitude"])), fill=(0,0,0), font=font5)
    # draw.text((680,45), "LATITUDE: ", fill=(0,0,0), font=font5)

    # draw.text((1130,45), format((groups[frame_num - 1]["longitude"])), fill=(0,0,0), font=font5)
    # draw.text((1020,45), "LONGITUDE: ", fill=(0,0,0), font=font5)

    frame = np.array(frame)

    font2 = ImageFont.truetype("font/Roboto-Bold.ttf", size=25)
    font3 = ImageFont.truetype("font/Roboto-Light.ttf", size=18)
    font4 = ImageFont.truetype("font/Depot Regular 400.ttf", size=20)
    font5 = ImageFont.truetype("font/Exo-Medium.ttf", size=17)
    font6 = ImageFont.truetype("font/Exo-Medium.ttf", size=14)
    # font5 = ImageFont.truetype('Terrabyte-Regular.ttf', size=15)

    frame = Image.fromarray(frame)
    draw = ImageDraw.Draw(frame, "RGBA")

    color = "rgb(0, 0, 0)"  # black color

    overlay = frame.copy()
    draw2 = ImageDraw.Draw(overlay)
    alpha = 0.5

    draw.rectangle((45, 91, 250, 790), fill=(92, 11, 249, 100), width=2)
    # draw.rectangle((45, 90,48,780), fill=(255,255,255, 255),width=2)
    # draw.rectangle((45, 87,375,140), fill=(0,0,0, 255),width=2)

    draw.rectangle((250, 91, 305, 790), fill=(131, 30, 239, 100), width=2)

    draw.rectangle((305, 91, 375, 790), fill=(162, 45, 230, 100), width=2)

    # draw.rectangle((45, 55,375,105), fill=(92,11,249, 150),width=2)

    # ------------------------------------
    # draw.rectangle((45, 91,250,720), fill=(92,11,249, 100),width=2)
    draw.rectangle((45, 90, 48, 790), fill=(255, 255, 255, 255), width=2)
    draw.rectangle((45, 87, 375, 90), fill=(0, 0, 0, 255), width=2)

    # draw.rectangle((250, 91,305,720), fill=(131,30,239, 100),width=2)

    # draw.rectangle((305, 91,375,720), fill=(162,45,230, 100),width=2)

    draw.rectangle((45, 55, 375, 85), fill=(92, 11, 249, 150), width=2)

    draw.text((70, 62), "VEHICLE TYPE", fill=(255, 255, 255), font=font4)
    draw.text((250, 62), "COUNT", fill=(255, 255, 255), font=font4)
    # draw.text((320, 62), "AREA", fill=(255, 255, 255), font=font4)


    global vehicle_count
    index_list = [idx for idx, val in enumerate(list(vehicle_count.values())) if val != 0]
    filtered_classes = [list(model.model.names.values())[x] for x in index_list]
    filtered_count = [list(vehicle_count.values())[x] for x in index_list]


    x = 60
    [
        draw.text(
            (x, 100 + i * 30),
            label,
            fill=(250, 250, 250),
            font=font6 if len(label) > 20 else font5,
        )
        # to show all classes names
        # for i, label in enumerate(list(model.model.names.values()))
        for i, label in enumerate(filtered_classes)
    ]
    x_off = 270
    [
        draw.text(
            (x_off, 100 + i * 30),
            str(v_count),
            fill=(255, 255, 255),
            font=font5,
        )
        # to show all classes count
        # for i,v_count in enumerate(list(vehicle_count.values()))
        for i,v_count in enumerate(filtered_count)
    ]


    frame = np.array(frame)
    return frame

In [13]:
def show_frames_into_video_final(frame):
    cv2.imshow(SOURCE_VIDEO, frame)
    if cv2.waitKey(5) & 0xFF == ord("q"):
        cv2.destroyAllWindows()

In [14]:
# def draw_labels(model, detections):
#         labels = [f"{tracker_id} {model.model.names[class_id]} {confidence:0.2f}"
#                 for _,_, confidence, class_id, tracker_id,_ 
#                 in detections]
#         # labels = [f"{model.names[class_id]} {confidence:0.2f}" for _,_,confidence, class_id,_,_ in detections]
#         return labels

def draw_labels(detections):
        labels = [f"{tracker_id} {model.model.names[class_id]} {confidence:0.2f}"
                for _,_, confidence, class_id, tracker_id,_ 
                in detections]
        # labels = [f"{model.names[class_id]} {confidence:0.2f}" for _,_,confidence, class_id,_,_ in detections]
        return labels

In [15]:
video_info = sv.VideoInfo.from_video_path(SOURCE_VIDEO)

In [16]:
from helpers.crossing_line import CrossingLine
import pandas as pd

In [17]:
crossing_line = CrossingLine()
cross_line = [np.array([[334, 710],[838, 726]])]
cross_line

[array([[334, 710],
        [838, 726]])]

In [18]:
# Anotating boundryboxes of the detected objects in an the input video
polygon_zones = [np.array([(196, 806), (596, 554), (896, 546), (796, 854)])]

polygon_zone_annotators = [
    PolygonZoneAnnotator(
        # color=Color.YELLOW,
        thickness=5,
        text_thickness=2,
        text_scale=1,
    )
    for index in range(len(polygon_zones))
]

box_annotators = [
    BoxAnnotator(
        # color=colors.by_idx(index),
        thickness=4,
        text_thickness=2,
        text_scale=0.7,
    )
    for index in range(len(polygon_zones))
]


def bounding_box_annotator(frame, detections):
    # box annotator without filtering and tracking
    for box_annotator in box_annotators:
        labels = [
            f"#{tracker_id} {model.names[class_id]} {confidence:0.2f}"
            for _, _, confidence, class_id, tracker_id, _ in detections
        ]
        frame = box_annotator.annotate(
            scene=frame, detections=detections, labels=labels
        )
    return frame


In [19]:

video_info = sv.VideoInfo.from_video_path(SOURCE_VIDEO)

In [20]:
def polygon_zones_interestd(frame, detections):
    # polygon zones annotator
    for zone_annotator in polygon_zone_annotators:
        labels = []
        mask = zone_annotator.trigger(
            polygon_zones=polygon_zones, detections=detections
        )
        filtered_zone_detections = detections[mask]

        # TODO below both zone annotations will be removed in production (zone boundary and interested zone should not be plotted but will be used for counting and area calculation)
        frame = zone_annotator.annotate(
            scene=frame, polygon_zones=polygon_zones, labels=labels, color=Color.YELLOW.as_bgr()
        )
        # frame = zone_annotator.annotate(
        #     scene=frame,
        #     polygon_zones=polygon_zones,
        #     labels=labels,
        #     color=Color.WHITE.as_rgb(),
        # )
    return frame, filtered_zone_detections

In [21]:
def bb_intersection_over_union(boxA, boxB):
	# determine the (x, y)-coordinates of the intersection rectangle
	xA = max(boxA[0], boxB[0])
	yA = max(boxA[1], boxB[1])
	xB = min(boxA[2], boxB[2])
	yB = min(boxA[3], boxB[3])
	# compute the area of intersection rectangle
	interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
	# compute the area of both the prediction and ground-truth
	# rectangles
	boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
	boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
	# compute the intersection over union by taking the intersection
	# area and dividing it by the sum of prediction + ground-truth
	# areas - the interesection area
	iou = interArea / float(boxAArea + boxBArea - interArea)
	# return the intersection over union value
	return iou

In [22]:
list_vehicle = [
        'Car',
        'Cycle',
        'MotorCycle',
        'Jeep',
        'Road_Roller',
        'Excavator',
        'Bus',
        'Three_wheeler',
        'Van',
        'Van_Ambulance',
        'G1_Pickup',
        'G2_Mini_Truck',
        'G10_2_Axle_Rigid_Truck',
        'G11_3_Axle_Rigid_Truck',
        'G12_Tandem_Axles_Truck',
        'Gx3_3_Axle_Articulated_Truck',
        'Gx4_4_Axle_Articulated_Truck',
        'Gx5_5_Axle_Articulated_Truck',
        'Gx6_6_Axle_Articulated_Truck',
        'LG1_Mini_Tanker',
        'LG10_2_Axle_Rigid_Tanker',
        'LG11_3_Axle_Rigid_Tanker',
        'LGx3_3_Axle_Articulated_Tanker',
        'LGx4_4_Axle_Articulated_Tanker',
        'LGx5_5_Axle_Articulated_Tanker',
        'LGx6_6_Axle_Articulated_Tanker',
        'GV10_2_Axle_Rigid_Motorbike_Carrier',
        'GV11_3_Axle_Rigid_Motorbike_Carrier',
        'GVx4_Articulated_Car_Carrier',
        'GV3W_Rigid_Wide_Bed',
        'GVx5_6w_Articulated_Wide_Bed',
        'RVxS_Super_Truck',
        'R1beG_Tractor_Trolley_Tanker'
]
dict_vehicle = {}


# def count_objects_(frame, detections):
#     tracking_id = detections.tracker_id
#     class_id = detections.class_id
#     if tracking_id is None:
#         return 
#     for i in range(len(tracking_id)):
#         print("Tracking ID",tracking_id[i])
#         print("Class ID",class_id[i])
#     return frame

In [23]:
crossing_line = CrossingLine()
cross_line = [np.array([[334, 710],[838, 726]])]
cross_line

[array([[334, 710],
        [838, 726]])]

In [24]:
CLASS_NAMES_DICT = model.model.names

In [25]:

def count_objects_(frame, objects_in_polygone_zone):

    global vehicle_tracking_ids, vehicle_count
    
    num_objects = len(objects_in_polygone_zone)
    print("Number of objects:", num_objects)
    print(objects_in_polygone_zone)
    tracking_id = objects_in_polygone_zone.tracker_id
    if tracking_id is not None:
        for i in range(len(objects_in_polygone_zone)):
            if objects_in_polygone_zone.tracker_id[i] not in vehicle_tracking_ids:
                vehicle_tracking_ids.append(objects_in_polygone_zone.tracker_id[i])
                vehicle_count[objects_in_polygone_zone.class_id[i]] += 1
        print('vehicle_tracking_ids: ', vehicle_tracking_ids)
        dev.print_dict(vehicle_count)

    cv2.putText(frame, f'Objects: {len(vehicle_tracking_ids)}', (500, 500), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2, cv2.LINE_AA)
    
    return frame

In [26]:
# vehicle_tracking_ids = []
# vehicle_count = {}
# def count_objects_(frame, objects_in_polygone_zone):

#     global vehicle_tracking_ids
    
#     num_objects = len(objects_in_polygone_zone)
#     print("Number of objects:", num_objects)
#     tracking_id = objects_in_polygone_zone.tracker_id
#     global temp1
#     temp1 = objects_in_polygone_zone
#     if tracking_id is not None:
#         for id in tracking_id:
#             if id not in vehicle_tracking_ids:
#                 vehicle_tracking_ids.append(id)
#                 print("vehicle_count: ",vehicle_tracking_ids)
#             print("Tracking ID",id)
#     # print("Tracking ID:", tracking_id)
#     cv2.putText(frame, f'Objects: {len(vehicle_tracking_ids)}', (500, 500), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2, cv2.LINE_AA)
    
#     return frame

In [27]:
def process_frame(frame: np.ndarray, _) -> np.ndarray:
    count = 0
    results = model.track(frame, show= True, persist=True)[0]
    detections = sv.Detections.from_ultralytics(results)
    if detections.tracker_id is None:
        return frame
    frame, filtered_zone_detections = polygon_zones_interestd(frame, detections)
    frame = bounding_box_annotator(frame, filtered_zone_detections)


    frame = count_objects_(frame, filtered_zone_detections)

    # count += filtered_zone_detections.tracker_id

    # print(filtered_zone_detections, count)

    frame = crossing_line.annotate(frame, cross_line)

    # count_objects_in_zone(filtered_zone_detections, classes)
    frame = drawing_on_frame_after_detection(frame)

    # print("FRame: ", frame)
    # frame = box_annotator.annotate(scene=frame, detections=detections, labels=labels)
    # frame = zone_annotator.annotate(scene=frame)
    show_frames_into_video_final(frame)

    print('_______________________________________________________________________________________')
    return frame

In [28]:
sv.process_video(source_path=SOURCE_VIDEO,target_path= "_result.mp4", callback=process_frame)

from IPython import display
cv2.destroyAllWindows()


0: 384x640 8 Cars, 2 Vans, 1 G2(Mini Truck), 94.5ms
Speed: 3.0ms preprocess, 94.5ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)
Number of objects: 2
Detections(xyxy=array([[     496.43,      647.07,      627.65,      756.51],
       [      731.8,      597.01,      831.32,      684.04]], dtype=float32), mask=None, confidence=array([    0.92291,     0.90903], dtype=float32), class_id=array([0, 0]), tracker_id=array([2, 5]), data={})
vehicle_tracking_ids:  [2, 5]
<class 'dict'>
S.No: 	Key: 	Value:
______________________________
1: 	0: 	2
2: 	1: 	0
3: 	2: 	0
4: 	3: 	0
5: 	4: 	0
6: 	5: 	0
7: 	6: 	0
8: 	7: 	0
9: 	8: 	0
10: 	9: 	0
11: 	10: 	0
12: 	11: 	0
13: 	12: 	0
14: 	13: 	0
15: 	14: 	0
16: 	15: 	0
17: 	16: 	0
18: 	17: 	0
19: 	18: 	0
20: 	19: 	0
21: 	20: 	0
22: 	21: 	0
23: 	22: 	0
24: 	23: 	0
25: 	24: 	0
26: 	25: 	0
27: 	26: 	0
28: 	27: 	0
29: 	28: 	0
30: 	29: 	0
31: 	30: 	0
32: 	31: 	0
33: 	32: 	0
______________________________________________________________________

In [29]:
list_vehicle = [
        'Car',
        'Cycle',
        'MotorCycle',
        'Jeep',
        'Road_Roller',
        'Excavator',
        'Bus',
        'Three_wheeler',
        'Van',
        'Van_Ambulance',
        'G1_Pickup',
        'G2_Mini_Truck',
        'G10_2_Axle_Rigid_Truck',
        'G11_3_Axle_Rigid_Truck',
        'G12_Tandem_Axles_Truck',
        'Gx3_3_Axle_Articulated_Truck',
        'Gx4_4_Axle_Articulated_Truck',
        'Gx5_5_Axle_Articulated_Truck',
        'Gx6_6_Axle_Articulated_Truck',
        'LG1_Mini_Tanker',
        'LG10_2_Axle_Rigid_Tanker',
        'LG11_3_Axle_Rigid_Tanker',
        'LGx3_3_Axle_Articulated_Tanker',
        'LGx4_4_Axle_Articulated_Tanker',
        'LGx5_5_Axle_Articulated_Tanker',
        'LGx6_6_Axle_Articulated_Tanker',
        'GV10_2_Axle_Rigid_Motorbike_Carrier',
        'GV11_3_Axle_Rigid_Motorbike_Carrier',
        'GVx4_Articulated_Car_Carrier',
        'GV3W_Rigid_Wide_Bed',
        'GVx5_6w_Articulated_Wide_Bed',
        'RVxS_Super_Truck',
        'R1beG_Tractor_Trolley_Tanker'
]
# dict_vehicle = { i:0 for i in list_vehicle}

# Dictionary with car names as keys and initial values as None
dict_vehicle = {car_name: None for car_name in list_vehicle}

# Printing the dictionary
# print(dict_vehicle)

In [30]:
import numpy as np

# Creating a NumPy array
np_array = np.array([1, 2, 3, 4, 5, 2, 3])

# Creating an empty dictionary
my_dict = {}
seen_values = set()

# Adding NumPy array values to the dictionary
for value in np_array:
    if value not in seen_values:
        my_dict[f'key_{value}'] = value
        seen_values.add(value)

print(seen_values)


{1, 2, 3, 4, 5}
